# Creating The Database


In [1]:
import pymysql

# Connect to the database
connection = pymysql.connect(host='127.0.0.1',
                             user='root',
                             port=3306,
                             db='db',
                             password='password')

cursor = connection.cursor()

## Dropping Existing Tables
Generally a good first step for setting up a database is to remove everything that is already there. This avoids any potential conflicts on creations/inserts.

Therefore we first **DROP** all of the tables we'll be using if they exist.

In [2]:
"""Drop tables"""
tables = ['statuses', 'edgelist', 'users']
for table in tables:
    print(table)
    drop_query = f"""DROP TABLE IF EXISTS {table};"""
    cursor.execute(drop_query)
connection.commit()

statuses
edgelist
users


## Creating Our Tables
Once we've cleared things out, we can start creating the tables we'll be using.

We create a table by calling **CREATE TABLE *table_name*(*colls*)**. One thing to keep in mind is the use of **PRIMARY KEY(*coll*)**, this simply identifies the column that will be used as the primary key.

In [3]:
"""Create users table to track twitter users"""
make_user_table = """CREATE TABLE users(
                        user_id VARCHAR(255),                        
                        screen_name VARCHAR(255) CHARSET utf8mb4,
                        name VARCHAR(255) CHARSET utf8mb4,
                        description VARCHAR(255) CHARSET utf8mb4,
                        location VARCHAR(255) CHARSET utf8mb4,                                                
                        created_at DATE,                     
                        followers_count INT,
                        following_count INT,
                        statuses_count INT,
                        listed_count INT,                        
                        PRIMARY KEY(user_id));"""

cursor.execute(make_user_table)
connection.commit()

## Load User Data to Write to the MySQL Table

In [4]:
import pandas as pd
from datetime import datetime    

users = pd.read_csv('/Users/dankoban/Documents/EM6586_DB_Management/final_project_twitter_data/seed_account_info.csv')
users['description'] = users['description'].apply(lambda x: str(x).replace("'", " "))
users['name'] = users['name'].apply(lambda x: str(x).replace("'", " "))
users['location'] = users['location'].apply(lambda x: str(x).replace("'", " "))
users['created_at'] = users['created_at'].apply(lambda x: datetime.strptime(x, '%Y-%m-%dT%H:%M:%S.000Z'))
print(len(users))
users.head()

1090


,user_id,username,name,description,location,created_at,followers_count,following_count,tweet_count,listed_count
0,4322065997,thereal_truther,The Real Truther,Team Reality forever. Exposing the COVID denie...,California,2015-11-29 21:35:35,7240.0,5516.0,129950.0,86.0
1,404290687,crwriter1,Joyce 🐝 Bowen,*Child advocate by virtue of circumstance. *Wr...,nan,2011-11-03 18:19:19,11270.0,11296.0,315831.0,71.0
2,26307708,mcfunny,R Marcucio,"Every time a conspiracy theory absorbs energy,...","Hope, On the Horizon",2009-03-24 19:49:20,1812.0,822.0,93220.0,29.0
3,448609949,paul_kangas,Paul❤️ Kangas⚡️,☀️Solar Justice builds 1000-panel Solar School...,"California, USA",2011-12-28 05:36:48,573.0,590.0,75794.0,56.0
4,54886540,doritmi,(((Dorit Reiss))),"Professor of Law, UC Hastings, teaches torts, ...","Bay Area, CA",2009-07-08 12:55:57,9509.0,9698.0,240084.0,201.0


## Write Users to MySQL Users Table

In [5]:
for i in range(0, len(users)):
    insert_stmt = f"INSERT INTO users(user_id, screen_name, name, description, \
                                      location, created_at, followers_count, \
                                      following_count, statuses_count, listed_count) VALUES \
                  ('{users['user_id'][i]}', \
                   '{users['username'][i]}', \
                   '{users['name'][i]}', \
                   '{users['description'][i]}', \
                   '{users['location'][i]}', \
                   '{users['created_at'][i]}', \
                   '{users['followers_count'][i]}', \
                   '{users['following_count'][i]}', \
                   '{users['tweet_count'][i]}', \
                   '{users['listed_count'][i]}')"
    cursor.execute(insert_stmt)
    connection.commit()

## Create Statuses Table
All subsequent table creation queries will look relatively the same, the key difference is the **FOREIGN KEY(*coll*)** statement. This statement defines a relationship to another table (which is a hard rule - no insertions that don't meet constraints).

Notes:
 - **REFERENCES *table_name(coll in table)**: This details where the foreign relationship can be found
 - **ON DELETE CASCADE**: This isn't required, but states that if the foreign key is removed, also remove this record

In [6]:
"""Create statuses table to track user posts"""
make_statuses_table = """CREATE TABLE statuses(
                        status_id VARCHAR(255),                        
                        user_id VARCHAR(255),
                        author_screen_name VARCHAR(255),
                        in_reply_to_user_id VARCHAR(255),
                        conversation_id VARCHAR(255),                        
                        lang VARCHAR(255),                        
                        text TEXT CHARSET utf8mb4,
                        created_at DATE,
                        retweet_count INT,
                        reply_count INT,
                        like_count INT,
                        quote_count INT,
                        referenced_tweet_type VARCHAR(255),
                        referenced_tweet_id VARCHAR(255),
                        PRIMARY KEY (status_id, created_at),
                        FOREIGN KEY (user_id)
                            REFERENCES users(user_id)
                            ON DELETE CASCADE);"""

cursor.execute(make_statuses_table)
connection.commit()

## Load Statuses Data to Write to MySQL

In [7]:
activity = pd.read_csv('/Users/dankoban/Documents/EM6586_DB_Management/final_project_twitter_data/activity.csv')
activity['text'] = activity['text'].apply(lambda x: str(x).replace("'", " "))
activity['created_at'] = activity['created_at'].apply(lambda x: datetime.strptime(x, '%Y-%m-%dT%H:%M:%S.000Z'))
print(len(activity))

# extra filter because it appears as though I missed some user profile queries due to API rate limits
users_with_profile_info = users['user_id'].unique().tolist()
activity = activity[activity['author_id'].isin(users_with_profile_info)]
print(len(activity))

# extra filter beacuse it appears as though I double queried a few accounts 
activity = activity.drop_duplicates(['id', 'created_at'])
activity.reset_index(inplace = True, drop = False)
print(len(activity))
activity.head()


274973
211252
208585


,index,author_id,author_screen_name,id,conversation_id,text,created_at,lang,in_reply_to_user_id,geo,retweet_count,reply_count,like_count,quote_count,referenced_tweet_type,referenced_tweet_id
0,0,4322065997,thereal_truther,1370489381841104898,1370447737838317574,@RussellBoullion @emmahn6 @naomirwolf @Henning...,2021-03-12 21:38:25,en,1.362841e+18,NaN,0,0,0,0,replied_to,1.370488e+18
1,1,4322065997,thereal_truther,1370488997105913858,1370447737838317574,@RussellBoullion @emmahn6 @naomirwolf @Henning...,2021-03-12 21:36:54,en,1.362841e+18,NaN,0,0,0,0,quoted,1.367243e+18
2,2,4322065997,thereal_truther,1370488898292387840,1370447737838317574,@RussellBoullion @emmahn6 @naomirwolf @Henning...,2021-03-12 21:36:30,en,1.362841e+18,NaN,0,1,0,0,replied_to,1.370487e+18
3,3,4322065997,thereal_truther,1370485453107777539,1370447737838317574,@RussellBoullion @emmahn6 @naomirwolf @Henning...,2021-03-12 21:22:49,en,1.362841e+18,NaN,0,5,0,0,replied_to,1.370467e+18
4,4,4322065997,thereal_truther,1370484201955356672,1370419244580622340,@Science78560422 @stacey_rudin @aid_taz @docka...,2021-03-12 21:17:50,en,1.352630e+18,NaN,0,0,0,0,replied_to,1.370478e+18


## Write Statuses to MySQL Statuses Table

In [8]:
for i in range(0, len(activity)):
    insert_stmt = f"INSERT INTO statuses(status_id, user_id, author_screen_name,\
                                         in_reply_to_user_id, conversation_id,\
                                         lang, text, created_at, retweet_count,\
                                         reply_count, like_count, quote_count,\
                                         referenced_tweet_type, referenced_tweet_id) VALUES \
                  ('{activity['id'][i]}',\
                   '{activity['author_id'][i]}',\
                   '{activity['author_screen_name'][i]}',\
                   '{activity['in_reply_to_user_id'][i]}',\
                   '{activity['conversation_id'][i]}',\
                   '{activity['lang'][i]}',\
                   '{activity['text'][i]}',\
                   '{activity['created_at'][i]}',\
                   '{activity['retweet_count'][i]}',\
                   '{activity['reply_count'][i]}',\
                   '{activity['like_count'][i]}',\
                   '{activity['quote_count'][i]}',\
                   '{activity['referenced_tweet_type'][i]}',\
                   '{activity['referenced_tweet_id'][i]}')"
    cursor.execute(insert_stmt)
    connection.commit()

## Create Edge List Table

In [9]:
"""Create edge table to track mention, hashtag, and url network"""
make_edgelist_table = """CREATE TABLE edgelist(
                        user_id VARCHAR(255),
                        author_screen_name_from VARCHAR(255),                         
                        status_id VARCHAR(255),               
                        to_entity VARCHAR(255) CHARSET utf8mb4,
                        edge_type VARCHAR(255),                                                                                                          
                        PRIMARY KEY (user_id, to_entity, status_id, edge_type),
                        FOREIGN KEY (user_id)
                            REFERENCES users(user_id)
                            ON DELETE CASCADE);"""

cursor.execute(make_edgelist_table)
connection.commit()

## Load Edge List Data to Write to MySQL

In [12]:
el = pd.read_csv('/Users/dankoban/Documents/EM6586_DB_Management/final_project_twitter_data/el.csv')
el['to'] = el['to'].apply(lambda x: x.lower())
print(len(el))
el = el.drop_duplicates()
print(len(el))

# extra filter because it appears as though I missed some user profile queries due to API rate limits
el = el[el['author_id_from'].isin(users_with_profile_info)]
el.reset_index(inplace = True, drop = True)

el.head()

646166
646044


,author_id_from,author_screen_name_from,status_id,to,edge_type
0,4322065997,thereal_truther,1370464773138120705,antivax,hashtag
1,4322065997,thereal_truther,1369703378633121792,covid,hashtag
2,4322065997,thereal_truther,1369703378633121792,antivax,hashtag
3,404290687,crwriter1,1370479219508183047,investigation,hashtag
4,404290687,crwriter1,1370475694258597892,writing,hashtag


## Write Edge List to MySQL Edge List Table

In [13]:
for i in range(0, len(el)):
    insert_stmt = f"INSERT INTO edgelist(user_id, author_screen_name_from, \
                                        status_id, to_entity, edge_type) VALUES \
                  ('{el['author_id_from'][i]}', \
                   '{el['author_screen_name_from'][i]}',\
                   '{el['status_id'][i]}',\
                   '{el['to'][i]}',\
                   '{el['edge_type'][i]}')"    
    cursor.execute(insert_stmt)
    connection.commit()